In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
sys.path.append("..")
from src import *

2024-06-19 14:01:37.282145: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-19 14:01:37.346789: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 14:01:38.785328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
base = "/data/bionets" if "ramses" in os.uname()[1] else "/data_nfs/"
config_path = "/data/bionets/je30bery/melanoma_data/config.json" if "ramses" in os.uname()[1] else "/data_nfs/je30bery/melanoma_data/config.json"

In [6]:
data = get_data_csv(groups=["Melanoma", "Nevus"], high_quality_only=False, pfs=False, config_path=config_path)
data = data.reset_index().drop("index", axis=1)

In [9]:
with open(config_path, "r") as f:
    configs = json.load(f)
    dataset_statistics = configs["dataset_statistics"]
    checkpoint_path = configs["model_weights"]
    pretrained_model = configs["pretrained_model_path"]

with open(os.path.join(dataset_statistics, f'melanoma_means.json'), 'r') as fp:
    means = json.load(fp)
    markers = list(means.keys())

In [10]:
model = ResNet18_pretrained(indim=len(markers), checkpoint_path=checkpoint_path)
model.load_state_dict(t.load(pretrained_model, map_location="cpu"))

# model = model.cuda()
model = model.eval()


dl = t.utils.data.DataLoader(MelanomaData(markers, pretrain=True, data=data, mode="val", config_path=config_path), batch_size=1, shuffle=False)
it = iter(dl)
features = list()
labels = list()
preds = list()
while True:
    try:
        x, y, id = next(it)
        labels.append(y.detach().cpu().numpy()[0])
        preds.append(model(x))
    except StopIteration:
        break

In [7]:
preds = [p.item() for p in preds]

In [13]:
np.unique(data["Group"])

array(['Melanoma', 'Nevus'], dtype=object)

In [12]:
len(data)

182

In [10]:
print("------- Sample level -------")
print_metrics(data["PFS label"], predictions["Prediction"])
print("------- Histo ID level -------")
y_true, y_pred = get_histo_id_level_predictions(data, predictions)
print_metrics(y_true, y_pred)

------- Sample level -------


KeyError: 'PFS label'